In [1]:
import os
import pandas as pd
from time import sleep
from tqdm import tqdm

Downloads the list of players on the court in a given quarter in a box score.

Needed to join with substitutions and account for players on court for every possession.

In [2]:
from nba_api.stats.endpoints import boxscoretraditionalv2

In [3]:
def period_start_time(period):
    if period > 5:
        return (720 * 4 + (period - 5) * (5 * 60)) * 10
    return (720 * (period - 1)) * 10

In [4]:
def get_data(y, i, q, overwrite=False):
    fpath = f'boxscoretraditionalv2/{y}/{i}_q{q}.csv'
    if not os.path.exists(fpath) or overwrite:
        req = boxscoretraditionalv2.BoxScoreTraditionalV2(
            game_id = i,
            start_range = str(period_start_time(q) + 5),
            end_range = str(period_start_time(q+1) - 5),
            range_type = '2'
        )
        df = req.player_stats.get_data_frame()
        df.to_csv(fpath, index=False)
#         sleep(0.3)

In [5]:
for y in range(2020, 2010, -1):
    dest_folder = f'boxscoretraditionalv2/{y}/'
    if not os.path.exists(dest_folder):
        os.mkdir(dest_folder)
    games = pd.read_csv(f'leaguegamelog/{y}.csv', dtype=str)
    y_id_q_map = []
    for i in games.GAME_ID.unique():
        game = pd.read_csv(f'playbyplay/{y}/{i}.csv')
        max_q = game.PERIOD.max() if not pd.isnull(game.PERIOD.max()) else 0
        for q in range(1, max_q+1):
            y_id_q_map.append((y, i, q))
    for y, i, q in tqdm(y_id_q_map):
        get_data(y, i, q)

100%|██████████| 5013/5013 [1:43:15<00:00,  1.24s/it]  
